In [1]:
%matplotlib inline
# ---- settings ----
import json
Settings = json.load(open('../settings.txt'))
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import sys
sys.path.insert(0, '../'); sys.path.insert(0, '../samples')
import paf_loader
from os.path import join, isdir, isfile
import numpy as np
import cv2
from cselect import color as cs
from time import time
from reid import reid

from mvpose.data import umpm
from mvpose.algorithm.settings import get_settings, get_tracking_settings

from mvpose.geometry.camera import ProjectiveCamera


Calib = []


root = '/home/julian/Videos/Ohhun2/'

FRAME = 200

def get_frame(FRAME):
    
    rvec1 = np.array([0.16970309, -0.63595406, -0.02899049])
    rvec2 = np.array([-0.04594394,  2.47154486 , 1.05356694])
    rvec3 = np.array([0.11478388, -2.60716985, -0.43695601])
    rvec4 = np.array([  0.50248564, -0.07703968, -0.00756907])
    rvec5 = np.array([0.11986254, 2.8707059,  0.58209686])
    RVECS = [rvec1, rvec2, rvec3, rvec4, rvec5]

    tvec1 = np.array([0.52766722, 0.44539547, 0.76997935])
    tvec2 = np.array([-0.72519327, -1.56923138,  3.54766366])
    tvec3 = np.array([1.94478036, -0.81084625 , 5.06761023])
    tvec4 = np.array([-0.875338  ,  0.45527339 ,-0.08985231])
    tvec5 = np.array([ 0.429569 ,  -0.96227696,  3.29255734])
    TVECS = [tvec1, tvec2, tvec3, tvec4, tvec5]
    
    Ims = []
    for cid in range(1, 6):
        #fname = 'test1_cam%1d-011.mp4' % cid
        fname = '180801_%1d-007.mp4' % cid
        fname = join(root, fname)
        assert isfile(fname)

        cap = cv2.VideoCapture(fname)
        for f in range(FRAME):
            ret, frame = cap.read()
        
        im = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        #im = cv2.resize(im, None, fx=0.5, fy=0.5)
        Ims.append(im)

        fname_c = 'cam%1d.txt' % cid
        fname_c = join(root, fname_c)
        assert isfile(fname_c)
        param_intrinsic = np.loadtxt(fname_c)
        K = param_intrinsic[:9].reshape(3,3)
        dist = param_intrinsic[9:]

        h, w, _ = Ims[0].shape
        rvec = RVECS[cid-1]
        tvec = TVECS[cid-1]
        Calib.append(ProjectiveCamera(K, rvec, tvec, dist, w, h))
    return Ims, Calib

Im, Calib = get_frame(FRAME)
    

# fig = plt.figure(figsize=(16,12))
# for idx, im in enumerate(Im):
#     ax = fig.add_subplot(3,2,idx+1)
#     ax.axis('off')
#     ax.imshow(im)
    
# plt.show()

/home/julian/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
# for cam in Calib:
#     print(cam.get_C())

In [ ]:
# fig = plt.figure(figsize=(16,12))
# for idx, (im, cam) in enumerate(zip(Im, Calib)):
#     ax = fig.add_subplot(3,2,idx+1)
#     ax.axis('off')
#     ax.imshow(im, alpha=0.5)
    
#     pt3d = np.array([[0, 0, 0]], 'float32')
#     x, y = np.squeeze(cam.projectPoints(pt3d))
#     ax.scatter(y, x, s=100)

In [ ]:
# pt3d = np.array([
#     [0, 0, 0]
# ], 'float32')

# cid = 2

# cam = Calib[cid]

# print(cam.tvec)

# im = Im[cid]

# x, y = np.squeeze(cam.projectPoints(pt3d))

# # ==================
# rvec = np.array([[ 0.11478388],
#        [-2.60716985],
#        [-0.43695601]])

# # ==================

# pp = cv2.projectPoints(pt3d, rvec, cam.tvec, cam.K, cam.distCoef)[0]

# print(x, y)
# print(pp)

# fig = plt.figure(figsize=(16,12))
# ax = fig.add_subplot(111)

# ax.imshow(im, alpha=0.5)
# ax.scatter(x, y, s=50)

# plt.show()

In [2]:
from mvpose import pose
from mvpose.plot.limbs import draw_mscoco_human
from mvpose.algorithm import settings

frame = FRAME
heatmaps, pafs = paf_loader.load_confidence_map_and_paf('ohhun', 
                                                        Im,
                                                        frame,
                                                        dir='/tmp')

params = settings.get_settings(scale_to_mm=1000)

_start = time()
Debug, detections = pose.estimate_heuristic(Calib, heatmaps, pafs, 
                                  settings=params, debug=True)
_end = time()
print('\ntotal elapsed:', _end - _start)
print("# detections", len(detections))

step 1: elapsed 5.059795379638672
step 2: elapsed 0.011467218399047852
step 3: elapsed 0.011270523071289062
step 4: elapsed 1.1462109088897705
-------------------------------------------
Handle transitivity:
	Intra:		 0
	Intra(choice):	 0
	Inter:		 2
	Inter(choice):	 0
-------------------------------------------
	Time =  8  ms
	result: 0

	Total cost: 160.37330760089353

# valid points:		 17
# invalid points:	 0
step 5: elapsed 0.010216236114501953
(smart candidate selection) [
	Time =  3  ms
	result: 0

	Total cost: 1.5834537785762122
] (smart candidate selection)
step 7: elapsed 0.010486602783203125

total elapsed: 6.250902891159058
# detections 1


In [3]:
detections

[[array([ 0.63889536, -0.56536032,  2.28506826]),
  array([ 0.51764516, -0.46907046,  2.1360222 ]),
  array([ 0.69085302, -0.44869916,  2.08709783]),
  array([ 0.76095869, -0.14566382,  2.0503609 ]),
  array([ 0.89010468, -0.05766877,  2.31410504]),
  array([ 0.3470537 , -0.49696663,  2.19741374]),
  array([ 0.24490163, -0.21848393,  2.16799899]),
  array([0.14915145, 0.03413135, 2.19552554]),
  array([0.60041769, 0.13049472, 2.17344677]),
  array([0.59038559, 0.51255061, 2.20358105]),
  array([0.53152672, 0.91390565, 2.16913694]),
  array([0.40194443, 0.14683679, 2.23825679]),
  array([0.40041655, 0.48461423, 2.2482277 ]),
  array([0.39687792, 0.78884922, 1.96715428]),
  array([ 0.65934548, -0.59546292,  2.26994716]),
  array([ 0.61690594, -0.60545286,  2.30022836]),
  None,
  None]]

In [ ]:
colors = cs.lincolor(22)/255

fig = plt.figure(figsize=(16,12))
for cid, im in enumerate(Im):
    ax = fig.add_subplot(3, 2, cid+1)
    ax.axis('off')
    ax.imshow(im, alpha=0.5)
    h,w,_ = im.shape
    ax.set_xlim([0, w])
    ax.set_ylim([h, 0])
    cam = Calib[cid]
    
    peaks2d = Debug.candidates2d.peaks2d[cid]
    for jid, peaks2d in enumerate(peaks2d):
        for x, y, s in peaks2d:
            ax.scatter(x, y, color=colors[jid], s=s*20)
    
plt.show()

In [ ]:
# import mvpose.geometry.stereo as stereo

# jid = 17
# cam1 = 2
# cam2 = 1

# # ------------------------
# def draw_peaks(ax, cid, colors):
#     peaks2d = Debug.candidates2d.peaks2d[cid][jid]
#     for i, (x, y, s) in enumerate(peaks2d):
#         ax.scatter(x, y, color=colors[i], s=s*20)

# # ------------------------

# pts2d_1 = Debug.candidates2d.peaks2d_undistorted[cam1][jid][:, 0:2]
# pts2d_2 = Debug.candidates2d.peaks2d_undistorted[cam2][jid][:, 0:2]


    

# get_y = lambda a, b, c, x: (-c - a * x)/b

# NOSE_1_COLORS = ['black', 'red']
# NOSE_2_COLORS = ['blue', 'green']

# im1 = Im[cam1]
# im2 = Im[cam2]

# P1 = Calib[cam1].P
# P2 = Calib[cam2].P
# F = stereo.get_fundamental_matrix(P1, P2)

# epilines_1to2 = np.squeeze(
#     cv2.computeCorrespondEpilines(pts2d_1, 1, F))
# if len(epilines_1to2.shape) <= 1:
#     epilines_1to2 = np.expand_dims(epilines_1to2, axis=0)
    
# epilines_2to1 = np.squeeze(
#     cv2.computeCorrespondEpilines(pts2d_2, 2, F))
# if len(epilines_2to1.shape) <= 1:
#     epilines_2to1 = np.expand_dims(epilines_2to1, axis=0)

# fig = plt.figure(figsize=(16,12))

# ax = fig.add_subplot(1, 2, 1)
# ax.imshow(im1, alpha=0.5)
# ax.axis('off')
# draw_peaks(ax, cam1, NOSE_1_COLORS)

# for i, (a, b, c) in enumerate(epilines_2to1):
#     x1 = 0
#     y1 = get_y(a, b, c, x1)
#     x2 = 640
#     y2 = get_y(a, b, c, x2)

    
#     ax.plot([x1, x2], [y1, y2], color=NOSE_2_COLORS[i])


# ax = fig.add_subplot(1, 2, 2)
# ax.imshow(im2, alpha=0.5)
# ax.axis('off')
# draw_peaks(ax, cam2, NOSE_2_COLORS)

# for i, (a, b, c) in enumerate(epilines_1to2):
#     x1 = 0
#     y1 = get_y(a, b, c, x1)
#     x2 = 640
#     y2 = get_y(a, b, c, x2)
#     ax.plot([x1, x2], [y1, y2], color=NOSE_1_COLORS[i])
    

# plt.show()


In [ ]:
from mvpose.algorithm.candidate_selection import project_human_to_2d
from mvpose.plot.limbs import draw_mscoco_human

Humans = detections

fig = plt.figure(figsize=(16,12))

colors = ['red', 'blue', 'green']

for idx, cam in enumerate(Calib):
    ax = fig.add_subplot(3,2,1+idx)
    ax.axis('off')
    im = Im[idx]
    h,w,_ = im.shape
    ax.imshow(im, alpha=0.6)
    
    for pid, human in enumerate(Humans):
        draw_mscoco_human(ax, human, cam, color=colors[pid])
        
plt.show()